In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy.sparse import csc_matrix, csr_matrix, hstack
import time
import numpy as np
import pickle

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization,CuDNNLSTM,Activation,BatchNormalization,Lambda,Add,Multiply,Subtract
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D,Concatenate
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
from keras.preprocessing import text, sequence
from sklearn.metrics import log_loss,roc_auc_score
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
import re
from keras import backend as K
from keras.backend import square
import time
import gc
# To get learning rate


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_pickle(FILE.train_ori.value)
# train = train.iloc[:200000].copy()
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))
train_length = len(train)


X = pd.concat([train,test],sort=False)
print(X.shape)
X = X.reset_index(drop=True)

# X_clean = pd.read_csv('../../data/original/cleaned_data_price_final.csv')
# X_append = pd.read_pickle('../../data/nn_features/clean.pkl')



print(X.shape)
# assert np.sum(X_clean['instance_id'].values != X['instance_id'].values) == 0
# assert np.sum(X_append['instance_id'].values != X['instance_id'].values) == 0
# X['model'] = X_append['model_new'].values


ignore_columns = ['instance_id','time','click'] + ['creative_is_js', 'creative_is_voicead', 'app_paid']

train shape is: (1001650, 36)
test shape is: (40024, 35)
(1041674, 36)
(1041674, 36)


# PB 4236

In [3]:
# need_process_col = list(set(X.columns) - set(ignore_columns))
# X_ = X[need_process_col].copy()


# doc_col = ['user_tags','model']
# non_doc_col = [f for f in need_process_col if f not in doc_col]
# # doc_col = doc_col + []
# # non_doc_col = non_doc_col+['user_tags']
# counter = 0

# X_doc = X[doc_col].copy()
# for col in tqdm(non_doc_col):
#     X_[col] = le.fit_transform(X_[col].astype(str))
#     X_[col] = col + '_'+X_[col].astype(str)
    
# for col in tqdm(doc_col):
#     X_doc[col] = X_doc[col].astype(str)
    
    

    

In [3]:
need_process_col = list(set(X.columns) - set(ignore_columns))
X_ = X[need_process_col].copy()


doc_col = ['user_tags','model']
non_doc_col = [f for f in need_process_col if f not in doc_col]
# doc_col = doc_col + []
# non_doc_col = non_doc_col+['user_tags']
counter = 0

X_doc = X[doc_col].copy()
for col in tqdm(non_doc_col):
#     col = 'creative_id'
    test_values = set(X_[col].iloc[train_length:].astype(str).unique())
    train_values = set(X_[col].iloc[:train_length].astype(str).unique())
    intersection = train_values.intersection(test_values)
    out_liyer = list(train_values.union(test_values) - intersection)
    if len(out_liyer) > 0:
#         print(len(out_liyer))
        out_liyer_mapping = pd.Series(index=list(out_liyer),data=1)
        filtered = (X_[col].astype(str).map(out_liyer_mapping) == 1)
#         print('col:{}, size:{}'.format(col,np.sum(filtered)))
        X_.loc[filtered,col] = np.nan
        
        
    X_[col] = le.fit_transform(X_[col].astype(str))
    X_[col] = col + '_'+X_[col].astype(str)
    
for col in tqdm(doc_col):
    X_doc[col] = X_doc[col].astype(str)
    
    

    

100%|██████████| 2/2 [00:00<00:00,  6.40it/s]


In [4]:
def get_tok(X,col,train_length,d_filter='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    X[col] = X[col].astype(str)

    tok_all=text.Tokenizer(num_words=X[col].nunique(),lower=False,filters=d_filter)
    tok_all.fit_on_texts(list(X[col].values))

    tok_train=text.Tokenizer(num_words=X[col].iloc[:train_length].nunique(),lower=False,filters=d_filter)
    tok_train.fit_on_texts(list(X[col].iloc[:train_length].values))

    tok_test=text.Tokenizer(num_words=X[col].iloc[train_length:].nunique(),lower=False,filters=d_filter)
    tok_test.fit_on_texts(list(X[col].iloc[train_length:].values))
    word_intersection = set(tok_train.word_index.keys()).intersection(set(tok_test.word_index.keys()))

    self_index = {}
    count = 1
    for word in word_intersection:
        self_index[word] = count
        count+=1
    self_index['unknown'] = count
    print('max index is: {}'.format(count))

    for word in tok_all.word_index.keys():
        tok_all.word_index[word] = self_index.get(word,count)
    return tok_all

In [5]:
train_index = train.index
holdout_index = None

In [6]:
num_folds = 5
seed = 1001
train_index_list = []
val_index_list = []
folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
for t,v in folds.split(train.loc[train_index],train.loc[train_index,'click']):
    train_index_list.append(train.loc[train_index].iloc[t].index)
    val_index_list.append(train.loc[train_index].iloc[v].index)
    
# check = []
# for i in val_index_list:
#     check.extend(list(i))
# assert len(set(check+list(holdout_index))) == len(train)

In [7]:
train_fold_y = {}
val_fold_y = {}
if holdout_index is not None:
    holdout_y = train.loc[holdout_index,'click'].values
else:
    holdout_y = None
for fold in range(num_folds):
    train_fold_y[fold] = train.loc[train_index_list[fold],'click'].values
    val_fold_y[fold] = train.loc[val_index_list[fold],'click'].values

In [8]:
info_dict = {}
train_all_dict = {}
train_fold_dict = {}

val_fold_dict = {}

holdout_input_dict = {}
test_input_dict = {}
maxlen = 1
prefix_input_nonDoc = 'input_'
prefix_input_Doc = 'input_rnn_'
for col in tqdm(non_doc_col):

    maxlen = 1
    tok=text.Tokenizer(num_words=X_[col].nunique(),lower=False,filters='@')
    tok.fit_on_texts(list(X_[col]))
    info_dict.update({prefix_input_nonDoc+col:{'tok':tok}})
    t = tok.texts_to_sequences(list(X_[col].iloc[:train_length].values))
    te = tok.texts_to_sequences(list(X_[col].iloc[train_length:].values))
    train_all_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    if holdout_index is not None:
        holdout_input_dict[prefix_input_nonDoc+col] = train_all_dict[prefix_input_nonDoc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_nonDoc+col: train_all_dict[prefix_input_nonDoc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_nonDoc+col:train_all_dict[prefix_input_nonDoc+col][list(val_index_list[fold])]})
        
sequence_size_dict = {}
for col in tqdm(doc_col):
    if col == 'user_tags':
        maxlen = 50
        tok = get_tok(X_doc,col=col,train_length=train_length,d_filter=',')
#         tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False, filters=',')
#         tok.fit_on_texts(list(X_doc[col]))

        
    elif col == 'model':
        maxlen = 15
#         tok = get_tok(X_doc,col=col,train_length=train_length)
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False)
        tok.fit_on_texts(list(X_doc[col]))
    else:
        maxlen = 15
#         tok = get_tok(X_doc,col=col,train_length=train_length)
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False)
        tok.fit_on_texts(list(X_doc[col]))
    info_dict.update({prefix_input_Doc+col:{'tok':tok}})
    sequence_size_dict[col] = maxlen
    t = tok.texts_to_sequences(list(X_doc[col].iloc[:train_length].values))
    te = tok.texts_to_sequences(list(X_doc[col].iloc[train_length:].values))
    train_all_dict[prefix_input_Doc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_Doc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    if holdout_index is not None:
        holdout_input_dict[prefix_input_Doc+col] = train_all_dict[prefix_input_Doc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_Doc+col: train_all_dict[prefix_input_Doc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_Doc+col:train_all_dict[prefix_input_Doc+col][list(val_index_list[fold])]})

  0%|          | 0/2 [00:00<?, ?it/s]

max index is: 1313


100%|██████████| 2/2 [01:00<00:00, 30.04s/it]


# Build NN model only use model

# 0.41479

In [ ]:
print('get second order x')
        ebd_dense_list = []
        print('project each in concat_list to a dense layer respectively')
        for each in concat_list:
            x_e =  Dense(MLP_unit, activation='relu')(each)
            x_e = Dropout(0.5)(x_e)
            ebd_dense_list.append(x_e)
        print('interaction with the dense layers')
        x_add = Add()(ebd_dense_list)
        x_inter1 = Multiply()([x_add,x_add])
        multi_list = []
        for each in ebd_dense_list:
            multi_list.append(Multiply()([each,each]))
        x_inter2 = Add()(multi_list)
        x_second = Subtract()([x_inter1,x_inter2])
        x_second = Dropout(0.5)(x_second)

In [56]:
def selfpool(x,dim=1):
    if dim == 1:
        def pol(x,n):
            return x[:,n,:]
    elif dim == 2:
        def pol(x,n):
            return x[:,:,n]
    else:
        raise ValueError('dim {} not support'.format(dim))
    f_pol = Lambda(pol,arguments={'n'})
    l = int(x.shape[dim])
    inter_list = []
    for index in range(l):
        f_pol = Lambda(pol,arguments={'n':index})
        inter_list.append(f_pol(x))
    x_add = Add()(inter_list)
    x_inter1 = Multiply()([x_add,x_add])
    multi_list = []
    for each in inter_list:
        multi_list.append(Multiply()([each,each]))
    x_inter2 = Add()(multi_list)
    x_second = Subtract()([x_inter1,x_inter2])
    return x_second
    
    

def get_nn_model(cols,doc_cols=[],numu_cols=[]):
    """
    cols, used to do ebd and dense layers
    doc_cols: used to do rnn
    there can be overlaps
    """
    input_list = []
    concat_list = []
    numu_list = []
    for col in cols:
        max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=1,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_'+col)(cur_input)
        embed_layer = SpatialDropout1D(0.5)(embed_layer)
        x = Flatten()(embed_layer)
        input_list.append(cur_input)
        concat_list.append(x)
    for col in doc_cols:
        max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        input_shape = sequence_size_dict[col]
        cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=input_shape,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_rnn_'+col)(cur_input)
        x = SpatialDropout1D(0.5)(embed_layer)
        if col == 'user_tags':
            print('user_tags')
            x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
            x = Conv1D(25, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
            x_aveP = GlobalAveragePooling1D()(x)
            x_maxP = GlobalMaxPooling1D()(x)
            x = concatenate([x_aveP,x_maxP])
        else:
#             x = selfpool(x,dim=2)
#             print(x.shape)
#             x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
#             x = Flatten()(x)
            x = Conv1D(25, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
            x_aveP = GlobalAveragePooling1D()(x)
            x_maxP = GlobalMaxPooling1D()(x)
            x = concatenate([x_aveP,x_maxP])
#             x = Dense(128, activation='relu')(x)
#             x = Dropout(0.5)(x)
#             x = Dense(64, activation='relu')(x)
#             x = Dropout(0.3)(x)
        input_list.append(cur_input)
        concat_list.append(x)

    if len(numu_cols) > 0:
        print('add numu...')
        nu_shape = len(numu_cols)
        cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
        x = BatchNormalization()(cur_input)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        input_list.append(cur_input)
        numu_list.append(x)
       
    if len(concat_list) > 1:
        x = concatenate(concat_list)
#     x = BatchNormalization()(x)
    

    if len(numu_list)>0:
        x = concatenate([x]+numu_list)
#         x = BatchNormalization()(x)
        
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

#     x1 = Dense(512, activation='relu')(x)
#     x1 = Dropout(0.2)(x1)
    
#     x2 = concatenate([x,x1])
#     x2_ = Dense(128, activation='relu')(x2)
#     x2_ = Dropout(0.2)(x2_)
    
#     x3 = concatenate([x,x1,x2_])
#     x3_ = Dense(64, activation='relu')(x3)
#     x3_ = Dropout(0.2)(x3_)
    
#     x = concatenate([x,x1,x2_,x3_])
#     x = Dropout(0.5)(x)


    preds = Dense(1, activation="sigmoid")(x)
    model = Model(input_list, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model

In [57]:
model = train_each_fold(train_fold_dict[0],
                                val_fold_dict[0],
                                train_fold_y[0],
                                val_fold_y[0],
                                non_doc_col,
                                doc_col,
                                tolerance=30,
                                train_batch_size=30000)

user_tags
Epoch 1/1
200331/200331 [==============================] - 4s 18us/step
0.4405876414155443
0.7295258563222319
best logloss is: 0.4405876414155443
remainning trial is: 0/30
total epoch trained: 0
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.4262632521295027
0.7521730182832231
best logloss is: 0.4262632521295027
remainning trial is: 0/30
total epoch trained: 1
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.42305903859311544
0.7574877957774959
best logloss is: 0.42305903859311544
remainning trial is: 0/30
total epoch trained: 2
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.42156582668871134
0.7602021260593359
best logloss is: 0.42156582668871134
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.4212303521359181
0.7613525828948434
best logloss is: 0.4212303521359181
remainning trial is: 0/30
total epoch trained: 4
Epoch 1/1
20033

200331/200331 [==============================] - 1s 4us/step
0.4170829696920182
0.7678262562882827
best logloss is: 0.4161744386413022
remainning trial is: 2/30
total epoch trained: 29
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.4161366772973735
0.7674923288887061
best logloss is: 0.4161366772973735
remainning trial is: 0/30
total epoch trained: 30
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.416241047539593
0.7671410909568182
best logloss is: 0.4161366772973735
remainning trial is: 1/30
total epoch trained: 31
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.41706409099153635
0.7680241277425046
best logloss is: 0.4161366772973735
remainning trial is: 2/30
total epoch trained: 32
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.4164860736768754
0.7678060332682888
best logloss is: 0.4161366772973735
remainning trial is: 3/30
total epoch trained: 33
Epoch 1/1
200331/200331 [==========

KeyboardInterrupt: 

In [46]:
model = train_each_fold(train_fold_dict[0],
                                val_fold_dict[0],
                                train_fold_y[0],
                                val_fold_y[0],
                                non_doc_col,
                                doc_col,
                                tolerance=30,
                                train_batch_size=30000)

Epoch 1/1
200331/200331 [==============================] - 3s 15us/step
0.44435263369015127
0.7305303186330571
best logloss is: 0.44435263369015127
remainning trial is: 0/30
total epoch trained: 0
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.4254868407125474
0.7527701897075791
best logloss is: 0.4254868407125474
remainning trial is: 0/30
total epoch trained: 1
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.42282694798092885
0.756725486763627
best logloss is: 0.42282694798092885
remainning trial is: 0/30
total epoch trained: 2
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.42177119585871203
0.7581953283752709
best logloss is: 0.42177119585871203
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.4204347820032639
0.7595937579223765
best logloss is: 0.4204347820032639
remainning trial is: 0/30
total epoch trained: 4
Epoch 1/1
200331/200331 

200331/200331 [==============================] - 1s 4us/step
0.4162281248455081
0.7672435064045706
best logloss is: 0.4161194095745543
remainning trial is: 5/30
total epoch trained: 28
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.4164340475350506
0.767745790974257
best logloss is: 0.4161194095745543
remainning trial is: 6/30
total epoch trained: 29
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.416222573920551
0.7675813217680467
best logloss is: 0.4161194095745543
remainning trial is: 7/30
total epoch trained: 30
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.4169682972281847
0.7677529865504893
best logloss is: 0.4161194095745543
remainning trial is: 8/30
total epoch trained: 31
Epoch 1/1
200331/200331 [==============================] - 1s 4us/step
0.4163178406173104
0.7675681200621749
best logloss is: 0.4161194095745543
remainning trial is: 9/30
total epoch trained: 32
Epoch 1/1
200331/200331 [============

KeyboardInterrupt: 

In [41]:
model = train_each_fold(train_fold_dict[0],
                                val_fold_dict[0],
                                train_fold_y[0],
                                val_fold_y[0],
                                non_doc_col,
                                doc_col,
                                tolerance=30,
                                train_batch_size=30000)

(?, 50)
Epoch 1/1
200331/200331 [==============================] - 2s 12us/step
0.43977965604093977
0.7339098027688216
best logloss is: 0.43977965604093977
remainning trial is: 0/30
total epoch trained: 0
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.42587318110852546
0.7524068100533696
best logloss is: 0.42587318110852546
remainning trial is: 0/30
total epoch trained: 1
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.422990472554352
0.7566450518377805
best logloss is: 0.422990472554352
remainning trial is: 0/30
total epoch trained: 2
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.42221585641956094
0.758532063940224
best logloss is: 0.42221585641956094
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.42109741386250654
0.7593534836017288
best logloss is: 0.42109741386250654
remainning trial is: 0/30
total epoch trained: 4
Epoch 1/1
200331

200331/200331 [==============================] - 1s 3us/step
0.41688931386451344
0.7656081117155507
best logloss is: 0.4168574711862803
remainning trial is: 1/30
total epoch trained: 29
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.4167094241978728
0.7658157671453224
best logloss is: 0.4167094241978728
remainning trial is: 0/30
total epoch trained: 30
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.41701589007484036
0.7663359303026396
best logloss is: 0.4167094241978728
remainning trial is: 1/30
total epoch trained: 31
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.416704642979733
0.766417465390222
best logloss is: 0.416704642979733
remainning trial is: 0/30
total epoch trained: 32
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.41667035668086355
0.766496962041875
best logloss is: 0.41667035668086355
remainning trial is: 0/30
total epoch trained: 33
Epoch 1/1
200331/200331 [==========

200331/200331 [==============================] - 1s 3us/step
0.41666960670360725
0.7669484555111888
best logloss is: 0.41637641105618656
remainning trial is: 2/30
total epoch trained: 58
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.41620433833772746
0.7671130365584298
best logloss is: 0.41620433833772746
remainning trial is: 0/30
total epoch trained: 59
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.4164402337642076
0.7671052437918198
best logloss is: 0.41620433833772746
remainning trial is: 1/30
total epoch trained: 60
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.4166866660090178
0.7670405183798317
best logloss is: 0.41620433833772746
remainning trial is: 2/30
total epoch trained: 61
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.4175739367423465
0.7667643885922604
best logloss is: 0.41620433833772746
remainning trial is: 3/30
total epoch trained: 62
Epoch 1/1
200331/200331 [===

KeyboardInterrupt: 

In [39]:
model = train_each_fold(train_fold_dict[0],
                                val_fold_dict[0],
                                train_fold_y[0],
                                val_fold_y[0],
                                non_doc_col,
                                doc_col,
                                tolerance=30,
                                train_batch_size=30000)

(?, 17)
Epoch 1/1
200331/200331 [==============================] - 2s 11us/step
0.49339919530444026
0.673679950581753
best logloss is: 0.49339919530444026
remainning trial is: 0/30
total epoch trained: 0
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.44066223640252233
0.7413590133123868
best logloss is: 0.44066223640252233
remainning trial is: 0/30
total epoch trained: 1
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.42808033100569653
0.7531161448738048
best logloss is: 0.42808033100569653
remainning trial is: 0/30
total epoch trained: 2
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.42609187656596786
0.7564367685944917
best logloss is: 0.42609187656596786
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.4221139943302295
0.7580793807893577
best logloss is: 0.4221139943302295
remainning trial is: 0/30
total epoch trained: 4
Epoch 1/1
2003

200331/200331 [==============================] - 1s 3us/step
0.4172520734904193
0.7658968595679848
best logloss is: 0.41668054124428267
remainning trial is: 3/30
total epoch trained: 29
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.41698770629582405
0.765912440558642
best logloss is: 0.41668054124428267
remainning trial is: 4/30
total epoch trained: 30
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.4167269125953221
0.7660016102846545
best logloss is: 0.41668054124428267
remainning trial is: 5/30
total epoch trained: 31
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.4168028436595024
0.7661649024188402
best logloss is: 0.41668054124428267
remainning trial is: 6/30
total epoch trained: 32
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.41657825019321565
0.7661913127227483
best logloss is: 0.41657825019321565
remainning trial is: 0/30
total epoch trained: 33
Epoch 1/1
200331/200331 [====

200331/200331 [==============================] - 1s 3us/step
0.41634180601870924
0.7672921258466092
best logloss is: 0.41623250582964183
remainning trial is: 6/30
total epoch trained: 58
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.4163452524022979
0.7671762849326034
best logloss is: 0.41623250582964183
remainning trial is: 7/30
total epoch trained: 59
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.41618225733418435
0.7675482459580016
best logloss is: 0.41618225733418435
remainning trial is: 0/30
total epoch trained: 60
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.41681143842258467
0.767113670715871
best logloss is: 0.41618225733418435
remainning trial is: 1/30
total epoch trained: 61
Epoch 1/1
200331/200331 [==============================] - 1s 3us/step
0.41694288396301665
0.7669330960879519
best logloss is: 0.41618225733418435
remainning trial is: 2/30
total epoch trained: 62
Epoch 1/1
200331/200331 [==

KeyboardInterrupt: 

In [20]:
model = train_each_fold(train_fold_dict[0],
                                val_fold_dict[0],
                                train_fold_y[0],
                                val_fold_y[0],
                                non_doc_col,
                                doc_col,
                                tolerance=30,
                                train_batch_size=30000)

get second order x
project each in concat_list to a dense layer respectively
interaction with the dense layers
Epoch 1/1
598529/598529 [==============================] - 6s 9us/step
0.42736679567997976
0.7562519439802371
best logloss is: 0.42736679567997976
remainning trial is: 0/30
total epoch trained: 0
Epoch 1/1
598529/598529 [==============================] - 4s 7us/step
0.4261155568545741
0.7586742639007411
best logloss is: 0.4261155568545741
remainning trial is: 0/30
total epoch trained: 1
Epoch 1/1
598529/598529 [==============================] - 4s 7us/step
0.42107705933936457
0.7624200949357899
best logloss is: 0.42107705933936457
remainning trial is: 0/30
total epoch trained: 2
Epoch 1/1
598529/598529 [==============================] - 4s 7us/step
0.42234088590906976
0.7635232145830972
best logloss is: 0.42107705933936457
remainning trial is: 1/30
total epoch trained: 3
Epoch 1/1
598529/598529 [==============================] - 4s 7us/step
0.4199840713888361
0.764139734238642

KeyboardInterrupt: 

In [ ]:
# def get_nn_model(cols,doc_cols=[],numu_cols=[]):
#     """
#     cols, used to do ebd and dense layers
#     doc_cols: used to do rnn
#     there can be overlaps
#     """
#     MLP_unit=128
#     input_list = []
#     concat_list = []
#     numu_list = []
#     for col in cols:
#         max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
# #         max_feature = len(set(info_dict[prefix_input_nonDoc+col]['tok'].index_word.values()))
#         embed_size = int(np.log2(max_feature)/np.log2(1.5))
#         if embed_size< 2:
#             embed_size = 2
#         cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
#         embed_layer = Embedding(max_feature,
#                             embed_size,
#                             input_length=1,
#                             trainable=True,
#                             embeddings_regularizer=l2(0.0005),
#                             name='ebd_'+col)(cur_input)
#         embed_layer = SpatialDropout1D(0.5)(embed_layer)
#         x = Flatten()(embed_layer)
#         input_list.append(cur_input)
#         concat_list.append(x)
#     for col in doc_cols:
#         max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
# #         max_feature = len(set(info_dict[prefix_input_Doc+col]['tok'].index_word.values()))
#         embed_size = int(np.log2(max_feature)/np.log2(1.5))
#         if embed_size< 2:
#             embed_size = 2
#         input_shape = sequence_size_dict[col]
#         cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
#         embed_layer = Embedding(max_feature,
#                             embed_size,
#                             input_length=input_shape,
#                             trainable=True,
#                             embeddings_regularizer=l2(0.0005),
#                             name='ebd_rnn_'+col)(cur_input)
#         x = SpatialDropout1D(0.5)(embed_layer)
#         x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
#         x = Conv1D(25, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
#         x_aveP = GlobalAveragePooling1D()(x)
#         x_maxP = GlobalMaxPooling1D()(x)
#         x = concatenate([x_aveP,x_maxP])

#         concat_list.append(x)
#         input_list.append(cur_input)

#     if len(numu_cols) > 0:
#         print('add numu...')
#         nu_shape = len(numu_cols)
#         cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
#         x = BatchNormalization()(cur_input)
#         x = Dense(128, activation='relu')(x)
#         x = Dropout(0.2)(x)
#         input_list.append(cur_input)
#         numu_list.append(x)
       
#     if len(concat_list) > 1:
#         x = concatenate(concat_list)
# #     x = BatchNormalization()(x)
    

#     if len(numu_list)>0:
#         x = concatenate([x]+numu_list)
# #         x = BatchNormalization()(x)
        
#     x1 = Dense(512, activation='relu')(x)
#     x1 = Dropout(0.2)(x1)
    
#     x2 = concatenate([x,x1])
#     x2_ = Dense(128, activation='relu')(x2)
#     x2_ = Dropout(0.2)(x2_)
    
#     x3 = concatenate([x,x1,x2_])
#     x3_ = Dense(64, activation='relu')(x3)
#     x3_ = Dropout(0.2)(x3_)
    
#     x = concatenate([x,x1,x2_,x3_])
#     x = Dropout(0.5)(x)

#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model(input_list, preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
#     return model


In [ ]:
# def get_nn_model(cols,doc_cols=[],numu_cols=[]):
#     """
#     cols, used to do ebd and dense layers
#     doc_cols: used to do rnn
#     there can be overlaps
#     """
#     MLP_unit=128
#     input_list = []
#     concat_list = []
#     numu_list = []
#     for col in cols:
#         max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
# #         max_feature = len(set(info_dict[prefix_input_nonDoc+col]['tok'].index_word.values()))
#         embed_size = int(np.log2(max_feature)/np.log2(1.5))
#         if embed_size< 2:
#             embed_size = 2
#         cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
#         embed_layer = Embedding(max_feature,
#                             embed_size,
#                             input_length=1,
#                             trainable=True,
#                             embeddings_regularizer=l2(0.0005),
#                             name='ebd_'+col)(cur_input)
#         embed_layer = SpatialDropout1D(0.5)(embed_layer)
#         x = Flatten()(embed_layer)
#         input_list.append(cur_input)
#         concat_list.append(x)
#     for col in doc_cols:
#         max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
# #         max_feature = len(set(info_dict[prefix_input_Doc+col]['tok'].index_word.values()))
#         embed_size = int(np.log2(max_feature)/np.log2(1.5))
#         if embed_size< 2:
#             embed_size = 2
#         input_shape = sequence_size_dict[col]
#         cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
#         embed_layer = Embedding(max_feature,
#                             embed_size,
#                             input_length=input_shape,
#                             trainable=True,
#                             embeddings_regularizer=l2(0.0005),
#                             name='ebd_rnn_'+col)(cur_input)
#         x = SpatialDropout1D(0.5)(embed_layer)
#         x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
#         x = Conv1D(25, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
#         x_aveP = GlobalAveragePooling1D()(x)
#         x_maxP = GlobalMaxPooling1D()(x)
#         x = concatenate([x_aveP,x_maxP])

#         concat_list.append(x)
#         input_list.append(cur_input)

#     if len(numu_cols) > 0:
#         print('add numu...')
#         nu_shape = len(numu_cols)
#         cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
#         x = BatchNormalization()(cur_input)
#         x = Dense(128, activation='relu')(x)
#         x = Dropout(0.2)(x)
#         input_list.append(cur_input)
#         numu_list.append(x)
       
#     if len(concat_list) > 1:
#         x = concatenate(concat_list)
# #     x = BatchNormalization()(x)
    

#     if len(numu_list)>0:
#         x = concatenate([x]+numu_list)
# #         x = BatchNormalization()(x)
        
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.2)(x)
    
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.2)(x)

#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model(input_list, preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
#     return model


In [11]:
def train_each_fold(input_train_dict,input_val_dict,y_train,y_val,cols,doc_col=[],tolerance=30,train_batch_size=5000):
    model = get_nn_model(cols,doc_col)
    cur_to = 0
    best_logloss = None
    best_weights = None
    count = 0
    base_lr = 0.001
    while True:
        model.fit(input_train_dict, y_train, 
                  batch_size=train_batch_size, 
                  epochs=1,
                  verbose=1,
                  shuffle=True,
                  )
        preds = model.predict(input_val_dict,30000,verbose=1)
        logloss = log_loss(y_val,preds)
        roc = roc_auc_score(y_val,preds)
        print(logloss)
        print(roc)
        if best_logloss is None:
            best_logloss = logloss
            best_weights = model.get_weights()
        else:
            if best_logloss > logloss:
                best_logloss = logloss
                best_weights = model.get_weights()
                cur_to = 0
            else:
                cur_to +=1
        if cur_to == tolerance:
            break
        print('best logloss is: {}'.format(best_logloss))
        print('remainning trial is: {}/{}'.format(cur_to,tolerance))
        print('total epoch trained: {}'.format(count))
        count+=1
    model.set_weights(best_weights)
    return model
    

In [12]:
def nn_K_fold(train_fold_dict,
              val_fold_dict,
              train_fold_y,
              val_fold_y,
              test_input_dict,
              val_index_list,
              train_df,
              test_df,
              pred_col_name = 'predicted_score',
              holdout_input_dict=None,
              holdout_y=None,
              holdout_index_list=None,
              nondoc_cols=[],
              doc_cols=[],
              tolerance=30,
              train_batch_size=5000,
              preds_batch=5000):
    """
    train_fold_dict: format, key - foldNum, value - nn input 
    train_fold_y: label for train fold, fotmat, key -foldNum, value - label
    val_fold_dict: format, key - foldNum, value - nn input 
    val_fold_y: label for val fold, fotmat, key -foldNum, value - label
    test_input_dict: format, key - foldNum, value - nn input 
    holdout_input_dict: Noneable, if none, not using holdout
    holdout_y: Noneable, label for holdout
    cols: all cols used to do ebd
    doc_cols, cols that are used to do rnn
    val_index_list: cv, going to predict and generate oof
    holdout_index_list: going to predict on each fold
    train_df: 'dataframe which only has id columns, which will be used to store oof prediction'
    test_df: 'dataframe which only has id columns, which will be used to store test prediction'
    """
    train_df = train_df.copy()
    test_df = test_df.copy()
    n_fold = len(train_fold_dict)
    test_preds_list = []
    val_score_list = []
    hold_out_preds_list = []
    holdout_score_list = []
    train_score_list = []
    cv_score_list = []
    train_df[pred_col_name] = np.nan
    test_df[pred_col_name] = np.nan
    
    for fold in range(n_fold):
        print('start fold {}...'.format(fold))
        model = train_each_fold(train_fold_dict[fold],
                                val_fold_dict[fold],
                                train_fold_y[fold],
                                val_fold_y[fold],
                                nondoc_cols,
                                doc_cols,
                                tolerance=tolerance,
                                train_batch_size=train_batch_size)
        train_preds =  model.predict(train_fold_dict[fold],preds_batch,verbose=1)
        val_preds = model.predict(val_fold_dict[fold],preds_batch,verbose=1)
        train_loss = log_loss(train_fold_y[fold],train_preds)
        val_loss = log_loss(val_fold_y[fold],val_preds)
        train_df.loc[val_index_list[fold],pred_col_name] = val_preds
        test_preds = model.predict(test_input_dict,preds_batch,verbose=1)
        test_preds_list.append(test_preds)
        val_score_list.append(val_loss)
        train_score_list.append(train_loss)
        print('Fold {} finish! val loss: {}.'.format(fold,val_loss))
        if holdout_index_list is not None:
            ho_preds = model.predict(holdout_input_dict,preds_batch,verbose=1)
            ho_loss = log_loss(holdout_y,ho_preds)
            ho_roc = roc_auc_score(holdout_y,ho_preds)
            holdout_score_list.append(ho_loss)
            hold_out_preds_list.append(ho_preds)
            print('hold out loss: {}'.format(ho_loss))
        del model
        gc.collect()
        time.sleep(5)
            
    print('finish training... calculating evl matrix')
    test_preds_list = np.array(test_preds_list)
    hold_out_preds_list = np.array(hold_out_preds_list)
    test_preds_final = np.mean(test_preds_list,axis=0)
    cv_score_mean = np.mean(val_score_list)
    train_score_mean = np.mean(train_score_list)
    test_df[pred_col_name] = test_preds_final
    print('cv mean is: {}'.format(cv_score_mean))
    if holdout_index_list is not None:
        ho_preds_final = np.mean(hold_out_preds_list,axis=0)
        ho_loss_overall = log_loss(holdout_y,ho_preds_final)
        ho_roc_overall = roc_auc_score(holdout_y,ho_preds_final)
        train_df.loc[holdout_index_list,pred_col_name] = ho_preds_final
        print('holdout loss overall is: {}'.format(ho_loss_overall))
        print('holdout roc overall is: {}'.format(ho_roc_overall))
        return train_df,test_df,cv_score_mean,ho_loss_overall,train_score_mean
    else:
        return train_df,test_df,cv_score_mean,0.0,train_score_mean
    
        

In [ ]:
train_df = train[['instance_id']].copy()
test_df = test[['instance_id']].copy()

train_save,test_save,cv_,ho_,ta_ = nn_K_fold(train_fold_dict,
                                      val_fold_dict,
                                      train_fold_y,
                                      val_fold_y,
                                      test_input_dict,
                                      val_index_list,
                                      train_df,
                                      test_df,
                                      pred_col_name = 'predicted_score',
                                      holdout_input_dict=holdout_input_dict,
                                      holdout_y=holdout_y,
                                      holdout_index_list=holdout_index,
                                      nondoc_cols=non_doc_col,
                                      doc_cols=doc_col,
                                      tolerance=30,
                                      preds_batch=5000)

In [ ]:
model = train_each_fold(train_fold_dict[0],
                                val_fold_dict[0],
                                train_fold_y[0],
                                val_fold_y[0],
                                non_doc_col,
                                doc_col,
                                tolerance=30,
                                train_batch_size=5000)

In [ ]:
def save_oof(train_df,test_df,cv,ta,ho=None,file_name='',path='../../data/nn_ebd/'):
    try:
        report = pd.read_csv(path+'report.csv')
    except:
        print('no report found! generate a new one!')
        report = pd.DataFrame()
    new_record = pd.DataFrame({'ho':[ho],'cv':[cv],'train_mean':[ta],'file':[file_name]})
    report = pd.concat([report,new_record],sort=False)
    train_df.to_pickle(path+'train/'+file_name+'.pkl')
    print(train_df.shape)
    test_df.to_csv(path+'test/'+file_name+'.csv',index=False)
    print(test_df.shape)
    report.to_csv(path+'report.csv',index=False)
    print('done!')
    

In [ ]:
save_oof(train_save,test_save,cv_,ta_,ho_,file_name='oldModel_RNNModelTag_ebdNANExclude_TagNAExclude_noholdout')

In [ ]:
name_dict = {'new_modelMakeOsOSV_rnnEBD':
             [['model','make','os','osv']],
             'new_modelMakeOsOSV+modelAppidInnerslotCreateWCreativeH_rnnEBD':
             [['model','make','os','osv'],['model','app_id','inner_slot_id','creative_width', 'creative_height']],
             'new_modelOSV':[['model', 'osv']],'new_model_onlyuTagRNN_100patience':[],}


In [ ]:
 model = get_nn_model(non_doc_col,doc_col)

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
import numpy as np

In [ ]:
for i in range(10):
    print(np.random.randint(1000))

